<a href="https://colab.research.google.com/github/QoutiOussama13/GooseAI/blob/main/Gradio_demo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To-do :

*    make the Ui similar to the one on our miro space (keep a first page as a dummy login page to use the information in the pdf file ) for reference see : [link ](https://github.com/MoroccoAI/2023-GenAI-Hackathon/blob/main/SehhaTech/Source%20Code/DemoSehhatech.ipynb)

*   Fill the function of reading text image with the code implementation suing OpenAI API (whisper,GPT4)



In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size

In [ ]:
def add_text(history, text):
    global query, arabic_query
    history = history + [(text, None)]
    arabic_query = text
    query = darija_to_eng(text)
    return history, gr.update(value="", interactive=False)

def add_audio(history, audio):
    global query, arabic_query
    history = history + [(audio, None)]
    return audio, None
from PIL import Image
def add_image(history, file):
    global is_there_image
    img = Image.open(file.name)
    img.save("/content/image.png")
    history = history + [((file.name,), None)]
    is_there_image = True
    return history

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)

def back():
  return gr.update(visible=False),gr.update(visible=True),gr.update(visible=True),gr.update(visible=True),gr.update(visible=False),gr.update(visible=False)

def move_next_page(a,b,c,d):
  global patient_info1, patient_info2, patient_info3
  patient_info1 = a + " " + b
  patient_info2 = c
  gender_mapping = {
    "ذكر": "Male",
    "أنثى": "Female"
}
  patient_info3 = english_gender = gender_mapping.get(d)
  return gr.update(visible=False),gr.update(visible=True),gr.update(visible=True),gr.update(visible=True),gr.update(visible=False),gr.update(visible=False)


In [ ]:
import gradio as gr

css=""".gradio-container {
      text-align:center;
      }
      #chatbot{
        margin:auto;
        height:400px;
        width:700px;
      }
      #message{
        margin:auto;
        width:700px;
      }
      #page1{
        margin:auto;
        width:700px;
      }
      """
talking = """


        """
not_talking = """

        """

with gr.Blocks(css=css) as demo:
  html_block = gr.HTML(not_talking, visible=False)
  chatbot = gr.Chatbot(
      [],
      elem_id="chatbot",
      bubble_full_width=False,
      visible=False
  )


  with gr.Column(visible=False, elem_id="message") as page2:
      # Create a row layout
      with gr.Row():
          # Create a column layout with a scale factor of 0.7
          with gr.Column(scale=0.6):
              # Create a textbox element for user input
              txt = gr.Textbox(show_label=False,placeholder="Enter text)
          with gr.Column(scale=0.3, min_width=0):
              # Create an audio input element from the microphone
              audio_input = gr.Audio(sources="microphone", type="filepath", show_download_button=False, show_share_button=False)
          with gr.Column(scale=0.1, min_width=0):
              btn = gr.UploadButton("📤", file_types=["image"])
          #html block for output audio
          html = gr.HTML()
          html.visible = False

          audio_input.stop_recording(add_audio, [chatbot,audio_input], [chatbot,audio_input], queue=False).then(
              print('ok')
          )
          txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
                    print('ok')
          )
          txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
          file_msg = btn.upload(add_image, [chatbot, btn], [chatbot], queue=False)
          chatbot.like(vote, None, None)
      with gr.Row():
        pdf_btn = gr.Button("inciddent repport", size="sm", variant="primary")
        recommandation_btn = gr.Button("Call a", size="sm", variant="secondary")


  with gr.Group(visible=False) as recommendation_page:
    gr.HTML("""




        """)
    gr.Markdown(
        """
        # :Call a
        """)


    recommendation = gr.Markdown("")
    with gr.Row():
        back_btn2 = gr.Button("Back to convo", size="sm", variant="primary")

  with gr.Group(visible=False) as pdf_report:
    gr.HTML(not_talking)
    report_html = gr.HTML("""

  """)
    with gr.Row():
        back_btn = gr.Button("Back to convo", size="sm", variant="primary")

  with gr.Group(elem_id="page1") as page1:
        gr.HTML("""




        """)
        gr.Markdown(
        """

        **Fill the text with your info**
        """)
        info1 = gr.Textbox(label="Email", placeholder=" Email")
        info2 = gr.Textbox(label="Full Name", placeholder="أدخل Full Name")
        info3 = gr.Number(label="Age")
        info4 = gr.Radio(label="Gender", choices=["Male", "Female"])
        gr.Textbox(label=" Additionel info ", placeholder="Enter info")

        connection_btn = gr.Button("Start conversation !")
        connection_btn.click(move_next_page,inputs=[info1,info2,info3,info4], outputs=[page1, page2, chatbot,html_block, pdf_report,recommendation_page])
        back_btn.click(back,outputs=[page1, page2, chatbot,html_block, pdf_report,recommendation_page])
        back_btn2.click(back,outputs=[page1, page2, chatbot,html_block, pdf_report,recommendation_page])

demo.queue()
demo.launch(debug = True)

/usr/local/lib/python3.10/dist-packages/gradio/layouts/column.py:53: UserWarning: 'scale' value should be an integer. Using 0.6 will cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/layouts/column.py:53: UserWarning: 'scale' value should be an integer. Using 0.3 will cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/layouts/column.py:53: UserWarning: 'scale' value should be an integer. Using 0.1 will cause issues.
  warnings.warn(


ok
ok
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c2eb5dd9bc2edf878b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
